## ¿Qué es el Deep Learning


### Feed forward neural network

### Fully connected layers (FC)

In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Activation
from keras.optimizers import SGD
from keras.layers import Dense
from keras.utils import np_utils
from imutils import paths
import numpy as np
import argparse
import cv2
import os

Using TensorFlow backend.


In [2]:
def image_to_feature_vector(image, size=(32, 32)):
    # resize the image to a fixed size, then flatten the image into
    # a list of raw pixel intensities
    return cv2.resize(image, size).flatten()

Download data from https://www.kaggle.com/c/dogs-vs-cats/data
You have to login in Kaggle and accept the challenge rules

cd /Volumes/Data/Computer_Vision/kaggle_dogs_cats
unzip train.zip

In [3]:
# initialize the data matrix and labels list
data = []
labels = []
train_path='/Volumes/Data/Computer_Vision/kaggle_dogs_cats/train'

imagePaths = list(paths.list_images(train_path))
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label (assuming that our
    # path as the format: /path/to/dataset/{class}.{image_num}.jpg
    image = cv2.imread(imagePath)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    
    # construct a feature vector raw pixel intensities, then update
    # the data matrix and labels list
    features = image_to_feature_vector(image)
    data.append(features)
    labels.append(label)
    
    # show an update every 1,000 images
    if i > 0 and i % 1000 == 0:
        print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 1000/25000
[INFO] processed 2000/25000
[INFO] processed 3000/25000
[INFO] processed 4000/25000
[INFO] processed 5000/25000
[INFO] processed 6000/25000
[INFO] processed 7000/25000
[INFO] processed 8000/25000
[INFO] processed 9000/25000
[INFO] processed 10000/25000
[INFO] processed 11000/25000
[INFO] processed 12000/25000
[INFO] processed 13000/25000
[INFO] processed 14000/25000
[INFO] processed 15000/25000
[INFO] processed 16000/25000
[INFO] processed 17000/25000
[INFO] processed 18000/25000
[INFO] processed 19000/25000
[INFO] processed 20000/25000
[INFO] processed 21000/25000
[INFO] processed 22000/25000
[INFO] processed 23000/25000
[INFO] processed 24000/25000


In [4]:
# encode the labels, converting them from strings to integers
le = LabelEncoder()
labels = le.fit_transform(labels)
 
# scale the input image pixels to the range [0, 1], then transform
# the labels into vectors in the range [0, num_classes] -- this
# generates a vector for each label where the index of the label
# is set to `1` and all other entries to `0`
data = np.array(data) / 255.0
labels = np_utils.to_categorical(labels, 2)
 
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
print("[INFO] constructing training/testing split...")
(trainData, testData, trainLabels, testLabels) = train_test_split(
    data, labels, test_size=0.25, random_state=42)

[INFO] constructing training/testing split...


In [5]:
# define the architecture of the network
model = Sequential()
model.add(Dense(768, input_dim=3072, kernel_initializer="uniform", activation="relu"))
model.add(Dense(384, kernel_initializer="uniform", activation="relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

In [6]:
# train the model using SGD
print("[INFO] compiling model...")
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd, metrics=["accuracy"])
model.fit(trainData, trainLabels, epochs=50, batch_size=128, verbose=1)

[INFO] compiling model...


/Users/rmunoz/.pyenv/versions/3.6.1/envs/keras/lib/python3.6/site-packages/keras/models.py:837: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/50
18750/18750 [==============================] - 11s - loss: 0.6853 - acc: 0.5629    
Epoch 2/50
18750/18750 [==============================] - 9s - loss: 0.6652 - acc: 0.5975     
Epoch 3/50
18750/18750 [==============================] - 11s - loss: 0.6539 - acc: 0.6129    
Epoch 4/50
18750/18750 [==============================] - 9s - loss: 0.6443 - acc: 0.6275     
Epoch 5/50
18750/18750 [==============================] - 15s - loss: 0.6399 - acc: 0.6361    
Epoch 6/50
18750/18750 [==============================] - 18s - loss: 0.6369 - acc: 0.6335    
Epoch 7/50
18750/18750 [==============================] - 14s - loss: 0.6293 - acc: 0.6451    
Epoch 8/50
18750/18750 [==============================] - 10s - loss: 0.6270 - acc: 0.6478    
Epoch 9/50
18750/18750 [==============================] - 10s - loss: 0.6222 - acc: 0.6538    
Epoch 10/50
18750/18750 [==============================] - 13s - loss: 0.6184 - acc: 0.6559    
Epoch 11/50
18750/18750 [========================

In [7]:
# show the accuracy on the testing set
print("[INFO] evaluating on testing set...")
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss, accuracy * 100))

[INFO] evaluating on testing set...
6250/6250 [==============================] - 1s     
[INFO] loss=0.6906, accuracy: 62.4480%
